In [17]:
import zipfile
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [18]:
zf = zipfile.ZipFile("playground-series-s4e6.zip")
df_train = pd.read_csv(zf.open("train.csv"))
df_test = pd.read_csv(zf.open("test.csv"))

In [19]:
df_train.head(5)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [60]:
#One liner to code the final column that is the outcome of our model
y_coded_column = pd.DataFrame({"Target": np.argmax((pd.get_dummies(df_train["Target"], dtype= float)), axis=1)})

#Setting up variables for train/test split
y = y_coded_column
x = df_train.drop(["id", "Target"], axis=1)

print (y)


       Target
0           2
1           0
2           0
3           1
4           2
...       ...
76513       2
76514       2
76515       1
76516       0
76517       2

[76518 rows x 1 columns]


In [21]:
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=42)

In [55]:
#Testing for best parameters
space={'learning_rate': hp.quniform("learning_rate", 0.01, 20,0.01),
        'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0,
        'scale_pos_weight':hp.quniform('scale_pos_weight',1,100,1)
    }
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( x_train, y_train), ( x_test, y_test)]
    
    clf.fit(x_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(x_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

print("The best hyperparameters are : ","\n")
print(best_hyperparams)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                 
0.4446941975953999                                     
  1%|          | 1/100 [00:03<05:59,  3.63s/trial, best loss: -0.4446941975953999]

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                            
0.4445896497647674                                                                
  2%|▏         | 2/100 [00:07<05:57,  3.65s/trial, best loss: -0.4446941975953999]

...

SCORE:                                                                              
0.4435441714584422                                                                  
 98%|█████████▊| 98/100 [05:55<00:07,  3.81s/trial, best loss: -0.44532148457919496]

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                              
0.44427600627286984                                                                 
 99%|█████████▉| 99/100 [05:59<00:03,  3.83s/trial, best loss: -0.44532148457919496]

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                              
0.4434396236278097                                                                  
100%|██████████| 100/100 [06:02<00:00,  3.63s/trial, best loss: -0.44532148457919496]
The best hyperparameters are :  

{'colsample_bytree': 0.5651153103317633, 'gamma': 1.1175786832078596, 'learning_rate': 0.07, 'max_depth': 14.0, 'min_child_weight': 9.0, 'reg_alpha': 40.0, 'reg_lambda': 0.3567502256248526, 'scale_pos_weight': 23.0}


In [56]:
model = xgb.XGBClassifier(learning_rate =0.07,
    colsample_bytree=0.56,
    n_estimators=1000,
    early_stopping_rounds=50,
    max_depth=14,
    min_child_weight=9,
    gamma=1.11,
    subsample=0.65,
    reg_alpha=40,
    reg_lambda=0.3567,
    objective= 'multi:softprob',
    nthread=-1,
    scale_pos_weight=23,
    random_state=42)

In [57]:
model.fit(x_train, 
          y_train, 
          verbose=True, 
          eval_set=[(x_test, y_test)])

[0]	validation_0-mlogloss:1.04687
[1]	validation_0-mlogloss:0.99868
[2]	validation_0-mlogloss:0.95302
[3]	validation_0-mlogloss:0.91410
[4]	validation_0-mlogloss:0.88025
[5]	validation_0-mlogloss:0.84687
[6]	validation_0-mlogloss:0.81586
[7]	validation_0-mlogloss:0.78913
[8]	validation_0-mlogloss:0.76493
[9]	validation_0-mlogloss:0.74174
[10]	validation_0-mlogloss:0.72073
[11]	validation_0-mlogloss:0.70257
[12]	validation_0-mlogloss:0.68591
[13]	validation_0-mlogloss:0.67047
[14]	validation_0-mlogloss:0.65571
[15]	validation_0-mlogloss:0.64153
[16]	validation_0-mlogloss:0.62940
[17]	validation_0-mlogloss:0.61905
[18]	validation_0-mlogloss:0.60838
[19]	validation_0-mlogloss:0.59815
[20]	validation_0-mlogloss:0.58930
[21]	validation_0-mlogloss:0.58083
[22]	validation_0-mlogloss:0.57245
[23]	validation_0-mlogloss:0.56580
[24]	validation_0-mlogloss:0.55914
[25]	validation_0-mlogloss:0.55294
[26]	validation_0-mlogloss:0.54705
[27]	validation_0-mlogloss:0.54111
[28]	validation_0-mlogloss:0.5

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.56, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1.11, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.07, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=14, max_leaves=None,
              min_child_weight=9, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, nthread=-1,
              num_parallel_tree=None, ...)

In [58]:
#Check feature Importance
importances = model.feature_importances_
columns = x.columns
i = 0

while i< len(columns):
    print (f" The importance of feature '{columns[i]}' is {round(importances[i]*100, 2)}%.")
    i += 1

 The importance of feature 'Marital status' is 0.32%.
 The importance of feature 'Application mode' is 0.69%.
 The importance of feature 'Application order' is 0.3%.
 The importance of feature 'Course' is 0.7%.
 The importance of feature 'Daytime/evening attendance' is 0.43%.
 The importance of feature 'Previous qualification' is 0.32%.
 The importance of feature 'Previous qualification (grade)' is 0.33%.
 The importance of feature 'Nacionality' is 0.22%.
 The importance of feature 'Mother's qualification' is 0.47%.
 The importance of feature 'Father's qualification' is 0.32%.
 The importance of feature 'Mother's occupation' is 0.44%.
 The importance of feature 'Father's occupation' is 0.38%.
 The importance of feature 'Admission grade' is 0.39%.
 The importance of feature 'Displaced' is 0.31%.
 The importance of feature 'Educational special needs' is 0.34%.
 The importance of feature 'Debtor' is 1.44%.
 The importance of feature 'Tuition fees up to date' is 6.83%.
 The importance of f

In [59]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred))


Accuracy: 0.83

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.83      0.87      6287
           1       0.65      0.61      0.63      3746
           2       0.86      0.92      0.89      9097

    accuracy                           0.83     19130
   macro avg       0.80      0.79      0.80     19130
weighted avg       0.83      0.83      0.83     19130



In [61]:
x_test = df_test.drop(['id'], axis=1)

predictions = model.predict(x_test)

columns = ['Dropout', 'Enrolled', 'Graduate']

# Mapping dictionary
label_mapping = {i: column for i, column in enumerate(columns)}

# Map the numeric labels to column titles
predictions_mapped = np.array([label_mapping[label] for label in predictions])

predictions_mapped


array(['Dropout', 'Graduate', 'Graduate', ..., 'Dropout', 'Dropout',
       'Dropout'], dtype='<U8')

In [62]:
df_submission = pd.DataFrame({
    'id': df_test['id'],
    'NObeyesdad': predictions_mapped
})
df_submission

,id,NObeyesdad
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Enrolled
...,...,...
51007,127525,Dropout
51008,127526,Dropout
51009,127527,Dropout
51010,127528,Dropout


In [63]:
df_submission.to_csv('XGBoost_Submission_Hypertuned.csv', index=False)